In [1]:
import pandas as pd
import numpy as np
from arcgis.features import GeoAccessor, GeoSeriesAccessor
df = pd.read_csv('https://dataverse.harvard.edu/api/access/datafile/3641280?format=original&gbrecs=true')
df

sdf = pd.DataFrame.spatial.from_featureclass(r"G:\courses\qmsa\Chapter15\Chapter15.gdb\USA_Counties")
#sdf = sdf.loc[sdf.State=='Nevada']
sdf

,OBJECTID,COUNTYNS,GEOID,ALAND,AWATER,NAME,State,Population,FIPSm,FIPS_Long,SHAPE
0,1,00161526,01001,1.539590e+09,2.576932e+07,Autauga County,Alabama,51979,1001,1001,"{'rings': [[[892103.0034999996, -491104.972099..."
1,2,00161527,01003,4.117584e+09,1.133129e+09,Baldwin County,Alabama,184627,1003,1003,"{'rings': [[[780227.0498000011, -659416.858599..."
2,3,00161528,01005,2.291821e+09,5.086468e+07,Barbour County,Alabama,25456,1005,1005,"{'rings': [[[1026116.7998000011, -548829.26269..."
3,4,00161529,01007,1.612482e+09,9.287974e+06,Bibb County,Alabama,21471,1007,1007,"{'rings': [[[825907.0381000005, -437079.944299..."
4,5,00161530,01009,1.670042e+09,1.507746e+07,Blount County,Alabama,54165,1009,1009,"{'rings': [[[871020.2313000001, -319035.668100..."
...,...,...,...,...,...,...,...,...,...,...,...
3103,3104,01609192,56037,2.700506e+10,1.668873e+08,Sweetwater County,Wyoming,41421,56037,56037,"{'rings': [[[-1065669.2237999998, 606868.27380..."
3104,3105,01605083,56039,1.034956e+10,5.708796e+08,Teton County,Wyoming,21033,56039,56039,"{'rings': [[[-1118845.9924999997, 811305.34070..."
3105,3106,01605084,56041,5.391917e+09,1.634564e+07,Uinta County,Wyoming,19266,56041,56041,"{'rings': [[[-1158426.9031000007, 542688.18050..."
3106,3107,01605085,56043,5.797822e+09,1.075432e+07,Washakie County,Wyoming,7928,56043,56043,"{'rings': [[[-886599.7495000008, 798061.703999..."


In [2]:
df = df.fillna(value={'FIPS': -1})
df = df[(df['FIPS'] != -1)]
df = df.astype({'FIPS': 'int64'})
reps = df[(df['party'] == 'republican')]
dems = df[(df['party'] == 'democrat')]
demsClean = dems[['year', 'FIPS', 'candidatevotes']]
demsClean = demsClean.rename(columns={"candidatevotes": "demVotes"})  
repsClean = reps[['year', 'FIPS', 'candidatevotes', 'totalvotes']]
repsClean = repsClean.rename(columns={"candidatevotes": "repVotes"})
repsClean['id'] = repsClean['year'].astype(str) + "_" + repsClean['FIPS'].astype(str)
demsClean['id'] = demsClean['year'].astype(str) + "_" + demsClean['FIPS'].astype(str)
demsClean = demsClean.set_index("id")
demsClean = demsClean[['demVotes']]
repsClean = repsClean.set_index("id")
demsClean
joinedVotes = demsClean.join(repsClean)
joinedVotes['dem'] = joinedVotes['demVotes'] > joinedVotes['repVotes']
joinedVotes

,demVotes,year,FIPS,repVotes,totalvotes,dem
id,,,,,,
2000_1001,4942.0,2000,1001,11993.0,17208,False
2000_1003,13997.0,2000,1003,40872.0,56480,False
2000_1005,5188.0,2000,1005,5096.0,10395,True
2000_1007,2710.0,2000,1007,4273.0,7101,False
2000_1009,4977.0,2000,1009,12667.0,17973,False
...,...,...,...,...,...,...
2016_2037,2421.0,2016,2037,1938.0,5062,True
2016_2038,2758.0,2016,2038,1143.0,5095,True
2016_2039,3142.0,2016,2039,1405.0,5639,True


In [3]:
import arcpy
yearlist=joinedVotes['year'].unique().tolist()
yearlist

for year in yearlist:
    print(year)
    dateDF = joinedVotes.loc[joinedVotes.year==year]
    joinedDate = sdf.set_index('FIPS_Long').join(dateDF.set_index('FIPS'))
    outputLOC = r"G:\courses\qmsa\Chapter15\Chapter15.gdb\USA_Counties_" + str(year)
    joinedDate.spatial.to_featureclass(location=outputLOC)
    arcpy.analysis.Intersect(outputLOC, outputLOC + "_Intersect", "ALL", None, "LINE")
    arcpy.analysis.SpatialJoin(outputLOC + "_Intersect", outputLOC + "_Intersect", outputLOC + "_Intersect_Join", "JOIN_ONE_TO_ONE", "KEEP_ALL", r'NAME "NAME" true true false 54 Text 0 0,First,#,G:\courses\qmsa\Chapter15\Chapter15.gdb\USA_Counties_2000_Intersect,NAME,0,54;dem "dem" true true false 2 Short 0 0,First,#,G:\courses\qmsa\Chapter15\Chapter15.gdb\USA_Counties_2000_Intersect,dem,-1,-1;NAME_1 "NAME_1" true true false 54 Text 0 0,First,#,G:\courses\qmsa\Chapter15\Chapter15.gdb\USA_Counties_2000_Intersect,NAME,0,54;dem_1 "dem_1" true true false 2 Short 0 0,First,#,G:\courses\qmsa\Chapter15\Chapter15.gdb\USA_Counties_2000_Intersect,dem,-1,-1', "ARE_IDENTICAL_TO", None, '')
    arcpy.analysis.Select(outputLOC + "_Intersect_Join", outputLOC + "_Intersect_Join_Select", "NAME <> NAME_1 And dem IS NOT NULL And dem_1 IS NOT NULL")
    arcpy.management.AddField(outputLOC + "_Intersect_Join_Select", "JoinType", "SHORT", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
    arcpy.management.CalculateField(outputLOC + "_Intersect_Join_Select", "JoinType", "not(!dem! == !dem_1!)", "PYTHON3", '', "TEXT")

2000


ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000725: Output Feature Class: Dataset G:\courses\qmsa\Chapter15\Chapter15.gdb\USA_Counties_2000_Intersect already exists.
Failed to execute (Intersect).


In [14]:
newlist = []

for year in yearlist:
    dateDF = joinedVotes.loc[joinedVotes.year==year]
    joinedDate = sdf.set_index('FIPS_Long').join(dateDF.set_index('FIPS'))
    B = float(joinedDate["dem"].sum())
    N = float(joinedDate["dem"].count())
    W = N - B
    outputLOC = r"G:\courses\qmsa\Chapter15\Chapter15.gdb\USA_Counties_" + str(year) + "_Intersect_Join_Select"
    osdf = pd.DataFrame.spatial.from_featureclass(outputLOC)
    outputLOC2 = r"G:\courses\qmsa\Chapter15\Chapter15.gdb\USA_Counties_" + str(year) + "_Intersect"
    lsdf = pd.DataFrame.spatial.from_featureclass(outputLOC2)
    ldf = lsdf.groupby('GEOID').OBJECTID.agg(['count'])
    ldf['countm1'] = ldf['count'] - 1
    ldf['Llm1'] = ldf['count'] * ldf['countm1']
    L = float(ldf['count'].sum())
    sumL = float(ldf['Llm1'].sum())
    J = float(osdf["JoinType"].count())
    Obw = float(osdf["JoinType"].sum())
    Ebw = (2*J*B*W) / (N*(N-1))
    T1 = ((sumL*B*W)/(N*(N-1)))
    num2 = 4*(J*(J-1)-sumL)*B*(B-1)*W*(W-1)
    T2 = num2/(N*(N-1)*(N-2)*(N-3))
    SEbw = (Ebw+T1+T2-(Ebw*Ebw))**0.5
    Zb = (Obw - Ebw) / SEbw
    newlist.append((year, B, W, N, J, Obw, Ebw, SEbw, Zb))
    
outdf = pd.DataFrame(newlist, columns =['year', 'B', 'W', 'N', 'J', 'Obw', 'Ebw', 'SEbw', 'Zb']) 
outdf

,year,B,W,N,J,Obw,Ebw,SEbw,Zb
0,2000,668.0,2438.0,3106.0,8726.0,1855.0,2947.082626,36.354432,-30.039876
1,2004,579.0,2528.0,3107.0,8728.0,1678.0,2647.632247,34.033820,-28.490256
2,2008,869.0,2238.0,3107.0,8728.0,1968.0,3517.887017,40.606866,-38.168102
3,2012,690.0,2417.0,3107.0,8728.0,1674.0,3016.669759,36.874256,-36.412118
4,2016,484.0,2623.0,3107.0,8728.0,1417.0,2296.390179,31.235951,-28.153142
